In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
history=pd.read_csv('https://drive.google.com/uc?export=download&id=1SmFlAHKnaqPy-lp8H3ZPegqQpz6C6gUq', sep='\t')
users=pd.read_csv('https://drive.google.com/uc?export=download&id=16UVDTOX6Gli4amZnDsRrow11ywI-WHl2', sep='\t')
validate_answers=pd.read_csv('https://drive.google.com/uc?export=download&id=1Nz-z3heM8zhUYahcpcsQ7wXxYUo-Ud2B', sep='\t')
validate=pd.read_csv('https://drive.google.com/uc?export=download&id=121sJ7OJ-ZXKAElaXiNHNsuZ49Qmbd2ki', sep='\t')

In [11]:
history.head()

,hour,cpm,publisher,user_id
0,10,30.00,1,15661
1,8,41.26,1,8444
2,7,360.00,1,15821
3,18,370.00,1,21530
4,8,195.00,2,22148


In [12]:
users.head()

,user_id,sex,age,city_id
0,0,2,19,0
1,1,1,0,1
2,2,2,24,2
3,3,1,20,3
4,4,2,29,4


In [13]:
validate_answers.head()

,at_least_one,at_least_two,at_least_three
0,0.0430,0.0152,0.0073
1,0.0130,0.0000,0.0000
2,0.0878,0.0135,0.0000
3,0.2295,0.1295,0.0727
4,0.3963,0.2785,0.2270


In [14]:
validate.head()

,cpm,hour_start,hour_end,publishers,audience_size,user_ids
0,220.0,1058,1153,"7,17",1906,"12,44,46,50,58,71,93,122,134,143,176,184,187,1..."
1,312.0,1295,1301,"3,18",1380,"29,81,98,102,165,167,195,205,218,231,242,263,3..."
2,70.0,1229,1249,"1,2,3,9,15,21",888,"12,23,25,29,45,85,92,124,156,190,272,334,456,5..."
3,240.0,1295,1377,"1,14",440,"44,122,187,209,242,255,312,345,382,465,513,524..."
4,262.0,752,990,"1,3,7,8",1476,"15,24,30,43,50,53,96,105,159,168,181,190,196,2..."


In [9]:
def load_answers(answers_filename):
    return pd.read_csv(answers_filename, sep="\t")


def get_smoothed_log_mape_column_value(responses_column, answers_column, epsilon):
    return np.abs(np.log(
        (responses_column + epsilon)
        / (answers_column + epsilon)
    )).mean()


def get_smoothed_mean_log_accuracy_ratio(answers, responses, epsilon=0.005):
    log_accuracy_ratio_mean = np.array(
        [
            get_smoothed_log_mape_column_value(responses.at_least_one, answers.at_least_one, epsilon),
            get_smoothed_log_mape_column_value(responses.at_least_two, answers.at_least_two, epsilon),
            get_smoothed_log_mape_column_value(responses.at_least_three, answers.at_least_three, epsilon),
        ]
    ).mean()

    percentage_error = 100 * (np.exp(log_accuracy_ratio_mean) - 1)

    return percentage_error.round(
        decimals=2
    )

In [15]:
data = validate.copy()

In [16]:
def calculate_mean_age(user_ids_str, users_df):
    if pd.isna(user_ids_str) or user_ids_str == '':
        return np.nan

    user_ids = [int(uid) for uid in user_ids_str.split(',') if uid.strip().isdigit()]

    ages = users_df[users_df['user_id'].isin(user_ids)]['age']

    return ages.mean() if not ages.empty else np.nan

In [58]:
def colculate_mean_cpm(row, history_df):
  hour_start = row.hour_start
  hour_end = row.hour_end
  publishers = [int(uid) for uid in row.publishers.split(',') if uid.strip().isdigit()]

  filtered_df = history[(history['hour'].between(hour_start, hour_end)) & (history['publisher'].isin(publishers))]
  return filtered_df['cpm'].mean()

def colculate_median_cpm(row, history_df):
  hour_start = row.hour_start
  hour_end = row.hour_end
  publishers = [int(uid) for uid in row.publishers.split(',') if uid.strip().isdigit()]

  filtered_df = history[(history['hour'].between(hour_start, hour_end)) & (history['publisher'].isin(publishers))]
  return filtered_df['cpm'].median()

In [32]:
def calculate_men_proportion(user_ids_str, users_df):
    if pd.isna(user_ids_str) or user_ids_str == '':
        return np.nan

    user_ids = [int(uid) for uid in user_ids_str.split(',') if uid.strip().isdigit()]

    user_data = users_df[users_df['user_id'].isin(user_ids)]

    if len(user_data) == 0:
        return float('nan')

    men_count = (user_data['sex'] == 1).sum()

    return men_count / len(user_data)

In [21]:
data['mean_age'] = data['user_ids'].apply(lambda x: calculate_mean_age(x, users))

In [19]:
data['duration'] = data['hour_end'] - data['hour_start']

In [59]:
data['mean_cpm'] = data.apply(lambda x: colculate_mean_cpm(x, history),  axis=1)
data['median_cpm'] = data.apply(lambda x: colculate_median_cpm(x, history),  axis=1)

In [33]:
data['men_proportion'] = data['user_ids'].apply(lambda x: calculate_men_proportion(x, users))

In [60]:
data.head()

,cpm,hour_start,hour_end,publishers,audience_size,user_ids,mean_age,duration,mean_cpm,men_proportion,median_cpm
0,220.0,1058,1153,"7,17",1906,"12,44,46,50,58,71,93,122,134,143,176,184,187,1...",28.721406,95,257.176642,0.537775,135.00
1,312.0,1295,1301,"3,18",1380,"29,81,98,102,165,167,195,205,218,231,242,263,3...",28.611594,6,230.437206,0.532609,135.00
2,70.0,1229,1249,"1,2,3,9,15,21",888,"12,23,25,29,45,85,92,124,156,190,272,334,456,5...",29.055180,20,195.542574,0.532658,123.28
3,240.0,1295,1377,"1,14",440,"44,122,187,209,242,255,312,345,382,465,513,524...",27.763636,82,187.107180,0.545455,120.00
4,262.0,752,990,"1,3,7,8",1476,"15,24,30,43,50,53,96,105,159,168,181,190,196,2...",44.760840,238,180.024268,0.611789,102.20


In [72]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

X_train, X_test, y_train, y_test = train_test_split(data[['cpm', 'mean_cpm', 'median_cpm', 'duration', 'mean_age', 'men_proportion', 'audience_size', 'hour_start', 'hour_end']], validate_answers, test_size=0.3, random_state=42)

model_at_least_one = RandomForestRegressor(
    n_estimators=200,
    max_depth=15,
    random_state=42
)

model_at_least_two = RandomForestRegressor(
    n_estimators=200,
    max_depth=15,
    random_state=42
)

model_at_least_three = RandomForestRegressor(
    n_estimators=200,
    max_depth=15,
    random_state=42
)

model_at_least_one.fit(X_train, y_train['at_least_one'])
model_at_least_two.fit(X_train, y_train['at_least_two'])
model_at_least_three.fit(X_train, y_train['at_least_three'])

pred_at_least_one = model_at_least_one.predict(X_train)
pred_at_least_two = model_at_least_two.predict(X_train)
pred_at_least_three = model_at_least_three.predict(X_train)

In [73]:
responses = pd.DataFrame({
    'at_least_one': pred_at_least_one,
    'at_least_two': pred_at_least_two,
    'at_least_three': pred_at_least_three
})

In [75]:
print(get_smoothed_mean_log_accuracy_ratio(y_test, responses))

336.52
